In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import math
import os

In [2]:
#--------------------------------------question1-------------------------------

d = pd.read_csv('Tags.csv')
df = pd.read_csv('Posts.csv')

df_Q = df.loc[df['PostTypeId']==1]

tags_df = d[['Id','TagName']].copy()
tags_df['Count']=0

def process_tags(tags_string, tag_name):
 
    tags_list = tags_string.strip('|').split('|')

    for tag in tags_list:
        tag.strip()

    return tag_name in tags_list

for i,row in d.iterrows():

    tag_name = row['TagName']

    if tag_name:
        count = df_Q['Tags'].apply(lambda x: process_tags(x,tag_name)).sum()
        tags_df.at[i,'Count']=count
           
tags_df = tags_df[['Id','Count']]
sorted_df = tags_df.sort_values(by='Count',ascending=False)
Tags = sorted_df.reset_index(drop=True)
top_tag_3 = Tags.loc[0:2, ['Id', 'Count']]
print(top_tag_3)


df1=pd.read_csv('Posts.csv')
df1_A=df1.loc[df1['PostTypeId']==2]
df1_AU=df1_A[['OwnerUserId','ParentId']]
Answerers = df1_AU[['OwnerUserId','ParentId']].value_counts().reset_index().rename(columns={"OwnerUserId": "OwnerUserId", "ParentId": "ParentId", "count": "Count"})
Answerers = Answerers[['OwnerUserId']]
# Answerers = Answerers.value_counts().reset_index().rename(columns={"OwnerUserId": "Id", "count": "Count"})
# top_answerer_3 = Answerers.loc[0:2,['Id','Count']]
# print(top_answerer_3)
# print(Answerers.columns)

    Id  Count
0  609   5162
1  249   4931
2   76   4928


In [3]:
Answerers = Answerers.value_counts().reset_index().rename(columns={"OwnerUserId": "Id", "count": "Count"})
top_answerer_3 = Answerers.loc[0:2,['Id',0]]
print(top_answerer_3)
print(Answerers.columns)

         Id     0
0    9113.0  2838
1  177980.0  2318
2    1204.0  2042
Index(['Id', 0], dtype='object')


In [4]:
#--------------------------------question2--------------------------------

# experts = Answerers.loc[Answerers['Count']>=20]
experts = Answerers.loc[Answerers[0]>=20]
experts = experts.sort_values(by='Id',ascending=True)

tags = Tags.loc[Tags['Count']>=20]
tags = tags.sort_values(by='Id',ascending=True)

Posts_filtered = df1_A.loc[df1_A['OwnerUserId'].isin(experts['Id'])]
ParentId_posts= df_Q.loc[df_Q['Id'].isin(Posts_filtered['ParentId'])]

tags_filtered = d.loc[d['Id'].isin(tags['Id'])]
tag_identifier=tags_filtered.set_index('Id')['TagName'].to_dict()

#expert matrix
dfm = pd.DataFrame(index=experts['Id'], columns=tags['Id'])

for e in experts['Id']:
    exp_post_ans=Posts_filtered.loc[Posts_filtered['OwnerUserId']==e]
    que_to_exp_ans=ParentId_posts.loc[ParentId_posts['Id'].isin(exp_post_ans['ParentId'])]

    for t in tags['Id']:
        tag_name= str(tag_identifier.get(t))
        if tag_name:
            count = que_to_exp_ans['Tags'].apply(lambda x: process_tags(x, tag_name)).sum()
            if(count!=0):dfm.loc[e,t]=count

print(f"Dimension of the Expert Matrix: {dfm.shape}")


Dimension of the Expert Matrix: (1160, 973)


In [5]:
#---------------------------------question3-------------------------------

def function(value):
    if pd.isna(value):
        return value
    if value<15:
        value = math.floor(value/3)
    else:
        value = 5
    return value

#utility matrix
dfm1=dfm.apply(lambda x: x.map(function))

dfm1_sum = dfm1.sum().sum()
dfm1_row = dfm1.sum(axis=1)
dfm1_col = dfm1.sum()

print(f"Summation of the Utility Matrix: {dfm1_sum}")

print(f"Highest Row Sum: {np.max(dfm1_row)}")

print(f"Highest Column Sum: {np.max(dfm1_col)}") 

#train and test data
train = dfm1.iloc[0:986,0:827]
test = dfm1.iloc[986:1160,827:973]

train_sum = train.sum().sum()
print(f"Summation of train matrix: {train_sum}")

print(f"Dimension of test matrix: {test.shape}")

test_sum = test.sum().sum()
print(f"Summation Value of test matrix: {test_sum}")


Summation of the Utility Matrix: 41180.0
Highest Row Sum: 1162.0
Highest Column Sum: 1403.0
Summation of train matrix: 32047.0
Dimension of test matrix: (174, 146)
Summation Value of test matrix: 642.0


In [7]:
#--------------------------------------question4--------------
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr


# Calculate Item-Item Similarity Matrix
def calculate_item_similarity(matrix):

    return matrix.T.corr(method='pearson')
    

item_similarity_matrix = dfm1.corr(method='pearson')
user_similarity_matrix = dfm1.T.corr(method='pearson')


# Predict Rating for Item-Item (Simple Average)
def predict_rating_simple_avg(user_id, tag_id, N):

    if tag_id not in item_similarity_matrix.columns:
        return 0

    similar_tags = item_similarity_matrix[tag_id].nlargest(N+1).index[1:N+1]
    ratings = dfm1.loc[user_id, similar_tags]

    return ratings.mean() if not ratings.empty else 0

# Predict Rating for Item-Item (Weighted Average)
def predict_rating_weighted_avg(user_id, tag_id, N):

    if tag_id not in item_similarity_matrix.columns:
        return 0

    similar_tags = item_similarity_matrix[tag_id].nlargest(N+1).index[1:N+1]
    weights = item_similarity_matrix[tag_id][similar_tags]
    
    ratings = dfm1.loc[user_id, similar_tags]
    valid_ratings = ratings.notnull()

    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()

    return weighted_sum / weight_sum if weight_sum != 0 else 0

# Predict Rating for User-User (Simple Average)
def predict_rating_simple_avg_user(user_id, tag_id, N):

    if user_id not in user_similarity_matrix.index: # Check if user_id exists in the index
        return 0

    if tag_id not in dfm1.columns:
        return 0

    similar_users = user_similarity_matrix.loc[user_id].nlargest(N+1).index[1:N+1]
    ratings = dfm1.loc[similar_users, tag_id]

    return ratings.mean() if not ratings.empty else 0

# Predict Rating for User-User (Weighted Average)
def predict_rating_weighted_avg_user(user_id, tag_id, N):

    if user_id not in user_similarity_matrix.index: # Check if user_id exists in the index
        return 0

    if tag_id not in dfm1.columns:
        return 0

    similar_users = user_similarity_matrix.loc[user_id].nlargest(N+1).index[1:N+1]
    weights = user_similarity_matrix.loc[user_id, similar_users]

    ratings = dfm1.loc[similar_users, tag_id]
    valid_ratings = ratings.notnull()

    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()

    return weighted_sum / weight_sum if weight_sum != 0 else 0

# Evaluate RMSE for User-User
def evaluate_rmse_user(predict_function, N):
    
    actual_ratings = []
    predicted_ratings = []

    for user_id in test.index:

        for tag_id in test.columns:

            if not pd.isnull(test.loc[user_id, tag_id]):

                actual_ratings.append(test.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))

     # Convert to numpy arrays for filtering NaNs
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Filter out any NaN values in the arrays
    mask = ~np.isnan(actual_ratings) & ~np.isnan(predicted_ratings)
    actual_ratings = actual_ratings[mask]
    predicted_ratings = predicted_ratings[mask]

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))


# Evaluate RMSE for Item-Item
def evaluate_rmse(predict_function, N):

    actual_ratings = []
    predicted_ratings = []

    for user_id in test.index:

        for tag_id in test.columns:

            if not pd.isnull(test.loc[user_id, tag_id]):

                actual_ratings.append(test.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))


    # Convert to numpy arrays for filtering NaNs
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Filter out any NaN values in the arrays
    mask = ~np.isnan(actual_ratings) & ~np.isnan(predicted_ratings)
    actual_ratings = actual_ratings[mask]
    predicted_ratings = predicted_ratings[mask]

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

def display_results():

    N_values = [2, 3, 5]

    print("Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5")
    print("------------------------------------------------------------")

    lis = [("Item-Item", predict_rating_simple_avg, predict_rating_weighted_avg) 
            ,("User-User", predict_rating_simple_avg_user, predict_rating_weighted_avg_user)]


    for method, predict_simple, predict_weighted in lis:

        print(f"{method} | Simple Average | RMSE |", end=" ")

        for N in N_values:
            rmse = evaluate_rmse(predict_simple, N) if method == "Item-Item" else evaluate_rmse_user(predict_simple, N)
            print(f"{rmse:.4f} |", end=" ")


        print()

        print(f"{method} | Weighted Average | RMSE |", end=" ")

        for N in N_values:
            rmse = evaluate_rmse(predict_weighted, N) if method == "Item-Item" else evaluate_rmse_user(predict_weighted, N)
            print(f"{rmse:.4f} |", end=" ")

        

        print()

display_results()


Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5
------------------------------------------------------------
Item-Item | Simple Average | RMSE | 0.9741 | 0.7659 | 0.8546 | 
Item-Item | Weighted Average | RMSE | 0.8251 | 0.7415 | 0.5467 | 
User-User | Simple Average | RMSE | 0.8530 | 0.7999 | 0.7451 | 
User-User | Weighted Average | RMSE | 0.6942 | 0.6782 | 0.6553 | 
